### Базовое решение кейса "Улучшение качества видео - super resolution" 
### Кейсодержатель: RUTUBE
#### Описание решения: 
Задача Super Resolution (SR) - повышение разрешения изображений / видео с сохранением качества контента.

Приведенное базовое решение основано на алгоритмическом повышении разрешения при помощи интерполяции и улучшении качества  изображения нейронной сетью.

Однако данное решение не является единственным, существует большое количество разнообразных подходов, которые показывают лучшее качество на данной задаче. Про существующие методы решения задачи SR вы можете прочитать здесь: https://blog.paperspace.com/image-super-resolution/. 

Про baseline модель вы можете подробнее прочитать тут: https://arxiv.org/pdf/1501.00092.pdf.

![Baseline модель](SRCNN.png)

In [ ]:
import os
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
import cv2
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# фиксируем seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed = 42
seed_everything(seed)

# Модель

In [ ]:
# функция инициализации весов модели
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
    elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

# Датасет

Обучение модели SRCNN происходит покадрово, поэтому выберем для обучения 5000 кадров случайным образом из 1000 видео (по 5 кадров из каждого видео).

Создадим все необходимые папки, train_path - путь куда сохранятся кадры, video_path - путь к папке с исходными видео.

In [ ]:
# video_path = 'path'
# train_path = './train_frames'

# lr_path = os.path.join(train_path, 'lr')
# hr_path = os.path.join(train_path, 'hr')

# if not os.path.exists(train_path):
#     os.system(f'mkdir -p {train_path}')

# if not os.path.exists(lr_path):
#     os.system(f'mkdir -p {lr_path}')

# if not os.path.exists(hr_path):
#     os.system(f'mkdir -p {hr_path}')

In [ ]:
# files = os.listdir(video_path)
# pairs = []
# for f in files:
#     if f.endswith('_144.mp4'):
#         hr_name = f.split('_')[0] + '_480.mp4'
#         pairs += [(f, hr_name)]

In [ ]:
# n_frames = 5000
# size = int(n_frames // len(pairs))

# save_idx = 0
# for idx in tqdm(range(len(pairs))):
#     pair = pairs[idx]

#     lr = os.path.join(video_path, pair[0])
#     hr = os.path.join(video_path, pair[1])

#     lr_cap = cv2.VideoCapture(lr)
#     hr_cap = cv2.VideoCapture(hr)

#     lr_len = int(lr_cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     hr_len = int(hr_cap.get(cv2.CAP_PROP_FRAME_COUNT))

#     assert lr_len == hr_len

#     frames_idx = [i for i in range(lr_len)]
#     if size:
#         frames_idx = np.random.choice(frames_idx, size=size, replace=False)

#     tmp_idx = 0
#     while True:
#         success_lr, frame_lr = lr_cap.read()
#         success_hr, frame_hr = hr_cap.read()
#         if not success_lr or not success_hr:
#             break
#         if tmp_idx in frames_idx:
#             lr_save_path = os.path.join(lr_path, f'{save_idx}.jpg')
#             hr_save_path = os.path.join(hr_path, f'{save_idx}.jpg')
#             cv2.imwrite(lr_save_path, frame_lr)
#             cv2.imwrite(hr_save_path, frame_hr)
#             save_idx += 1
#         tmp_idx += 1

Данный класс формирует датасет для обучения / валидации и тестирования.

Структура датасета: корневая папка -> папки train / val / test -> в каждой папке train / val / test лежит 2 папки lr и hr, внутри папок лежат изображения в низком и высоком разрешениях соответственно. Названия файлов в папке lr и hr должны совпадать, например lr/frame1.jpg и hr/frame1.jpg будет использоваться как одно изображение в разных разрешениях для обучения модели.

In [ ]:
from dataset import SRDataset


Аугментации ниже используются для получения torch.FloatTensor с нужными размерами.

In [ ]:
from dataset import SameTransform

# Обучение

In [ ]:
from ESRGAN_train import ESRGAN_Trainer

In [ ]:
# создаем объект - trainer для запуска процесса обучения и инференса
trainer = ESRGAN_Trainer()

In [ ]:
# запускаем процесс обучения
trainer.train()

# Инференс

Задаем путь к видео низкого разрешения, которое лежит у нас на диске (lr_video) и путь к выходному видео, обработанному моделью в высоком разрешении (hr_video).

In [ ]:
lr_video = '/home/owner/Documents/DEV/Python/SuperResolution/rutube_hackaton_super_resolution_khabarovsk/train/1_144.mp4'
hr_video = '/home/owner/Documents/DEV/Python/SuperResolution/rutube_hackaton_super_resolution_khabarovsk/train/1_480_newmp4'

trainer.super_resolution(lr_video, hr_video)